In [ ]:
!pip install -U transformers
!pip install --upgrade transformers accelerate bitsandbytes
!pip install -q transformers datasets peft accelerate bitsandbytes

In [ ]:
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
from transformers import TrainingArguments, Trainer


In [ ]:
# JSON 파일 로드
with open("/content/project_dataset_20250610_030527.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# JSON이 리스트인지 확인
if isinstance(data, dict):
    data = data["data"]

# Dataset 생성
dataset = Dataset.from_list(data)

# Train/Test 분할
dataset_dict = dataset.train_test_split(test_size=0.1)
train_dataset = dataset_dict['train']
test_dataset = dataset_dict['test']

In [ ]:
tokenized_train = train_dataset.map(format_sample)
tokenized_test = test_dataset.map(format_sample)

In [ ]:

HF_TOKEN = "허깅페이스_토큰입력"

model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=HF_TOKEN,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=HF_TOKEN,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# pad_token 설정
tokenizer.pad_token = tokenizer.eos_token

def format_sample(sample):
    prompt = sample["prompt"]
    completion = sample["completion"]
    text = f"{prompt}\n{completion}"
    return tokenizer(text, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(format_sample)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_dir="./logs",
    output_dir="./mistral-project-lora",
    save_total_limit=1,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
)

# Trainer 정의 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

trainer.train()

In [ ]:
model.save_pretrained("mistral-project-lora")
tokenizer.save_pretrained("mistral-project-lora")

In [ ]:
!zip -r mistral-jaemini-lora.zip mistral-jaemini-lora
from google.colab import files
files.download("mistral-project-lora.zip")

###테스트

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# 경량화 설정 (4bit 로딩)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# 베이스 모델 (Mistral) 로딩
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto",  # GPU/CPU에 분산 로딩
    trust_remote_code=True
)

# LoRA 파인튜닝된 가중치 적용
model = PeftModel.from_pretrained(
    base_model,
    "/content/mistral-project-lora",  # 파인튜닝된 모델
    device_map="auto"
)
model.eval()

# 토크나이저 로딩 (원본과 동일한 것 사용해야 함)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", use_fast=True)

# 테스트 프롬프트
prompt = "오늘 방송 시작하겠습니다.\nAI:"

# 입력 토크나이즈
inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

# 생성
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

# 출력 디코딩
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)